# Xây Dựng & Đánh Giá Mô Hình

Notebook này huấn luyện các mô hình gợi ý và phân cụm, sau đó đánh giá kết quả.

## Recommendation Models:
1. **KNN Cosine (Full Features)**: KNN với cosine similarity trên tất cả đặc trưng
2. **KNN Cosine (Numeric Only)**: KNN với cosine similarity chỉ trên đặc trưng số
3. **KNN TF-IDF (Text)**: KNN với cosine similarity trên TF-IDF từ mô tả
4. **Content-Based Cosine Matrix**: Content-based filtering với ma trận cosine similarity (tính cosine_similarity trực tiếp từ sklearn.metrics.pairwise)
5. **Gensim Doc2Vec** ⭐: Content-based filtering với Gensim Doc2Vec embeddings (theo yêu cầu đề bài)

## Clustering Models:
1. **KMeans**: Phân cụm partitioning với k=5
2. **Gaussian Mixture Model (GMM)**: Phân cụm probabilistic
3. **Agglomerative Clustering**: Phân cụm hierarchical với linkage Ward


In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
)
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.utils import simple_preprocess
import warnings
warnings.filterwarnings("ignore")

DATA_PATH = "/Users/doananh/Documents/Documents - Doan’s MacBook Pro/đồ án DS/project2/data_motobikes.xlsx - Sheet1.csv"
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)


def parse_price(value: str) -> float | None:
    if not value or pd.isna(value):
        return None
    value = str(value).lower()
    if value in {"đang cập nhật", "liên hệ", "thỏa thuận"}:
        return None
    value = value.replace("triệu", "tr").replace("tỷ", "ty")
    value = value.replace("đ", "").replace("vnđ", "").replace("vnd", "")
    value = value.replace(" ", "")
    match = re.search(r"([0-9]+(?:[\.,][0-9]+)?)", value)
    if not match:
        return None
    number = match.group(1).replace(",", ".")
    try:
        number = float(number)
    except ValueError:
        return None
    if value.endswith("ty"):
        return number * 1000.0
    return number if value.endswith("tr") else number / 1_000_000


def parse_km(value: str) -> float | None:
    if not value or pd.isna(value):
        return None
    value = str(value).lower().replace("km", "")
    value = value.replace(",", "").replace(" ", "").replace(".", "")
    match = re.search(r"(\d+)", value)
    return float(match.group(1)) if match else None


df = pd.read_csv(DATA_PATH)

base_cols = [
    "Giá", "Khoảng giá min", "Khoảng giá max", "Năm đăng ký", "Số Km đã đi",
    "Thương hiệu", "Dòng xe", "Loại xe", "Dung tích xe", "Tình trạng"
]

text_cols = ["Tiêu đề", "Mô tả chi tiết"]

# --- Data cleansing ---------------------------------------------------------
df_model = df[base_cols + ["id", "Tiêu đề", "Mô tả chi tiết"]].copy()

for col in ["Giá", "Khoảng giá min", "Khoảng giá max"]:
    df_model[col] = df_model[col].apply(parse_price)

df_model["Năm đăng ký"] = pd.to_numeric(df_model["Năm đăng ký"], errors="coerce")
df_model["Số Km đã đi"] = df_model["Số Km đã đi"].apply(parse_km)

df_model = df_model.dropna(subset=base_cols).reset_index(drop=True)

numeric_cols = ["Giá", "Khoảng giá min", "Khoảng giá max", "Năm đăng ký", "Số Km đã đi"]
cat_cols = ["Thương hiệu", "Dòng xe", "Loại xe", "Dung tích xe", "Tình trạng"]

# Feature matrix (numeric + categorical one-hot)
preprocess_all = ColumnTransformer([
    ("num", StandardScaler(), numeric_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
])

X_all = preprocess_all.fit_transform(df_model[base_cols])

# Numeric-only matrix
scaler_numeric = MinMaxScaler()
X_numeric = scaler_numeric.fit_transform(df_model[numeric_cols])

# Text TF-IDF matrix
text_series = (
    df_model[text_cols]
    .fillna("")
    .agg(" ".join, axis=1)
)
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_text = vectorizer.fit_transform(text_series)

# Gensim Doc2Vec embeddings
tagged_docs = [
    TaggedDocument(words=simple_preprocess(text, min_len=2), tags=[str(i)])
    for i, text in enumerate(text_series)
]
doc2vec_model = Doc2Vec(vector_size=100, window=5, min_count=2, workers=4, epochs=20, seed=RANDOM_SEED)
doc2vec_model.build_vocab(tagged_docs)
doc2vec_model.train(tagged_docs, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
X_gensim = np.array([doc2vec_model.dv[str(i)] for i in range(len(text_series))])

# --- Recommendation models -------------------------------------------------
recommenders = {
    "knn_cosine_all": {
        "description": "KNN (cosine) trên đặc trưng số + one-hot",
        "model": NearestNeighbors(metric="cosine", n_neighbors=6),
        "features": X_all,
        "type": "knn",
    },
    "knn_cosine_numeric": {
        "description": "KNN (cosine) chỉ với đặc trưng số chuẩn hóa",
        "model": NearestNeighbors(metric="cosine", n_neighbors=6),
        "features": X_numeric,
        "type": "knn",
    },
    "knn_tfidf_text": {
        "description": "KNN (cosine) trên TF-IDF tiêu đề + mô tả",
        "model": NearestNeighbors(metric="cosine", n_neighbors=6, algorithm="brute"),
        "features": X_text,
        "type": "knn",
    },
    "content_based_cosine_matrix": {
        "description": "Content-based filtering với cosine similarity matrix",
        "model": None,  # Dùng cosine_similarity trực tiếp
        "features": X_all,
        "type": "cosine_matrix",
    },
    "gensim_doc2vec": {
        "description": "Gensim Doc2Vec embeddings với cosine similarity",
        "model": NearestNeighbors(metric="cosine", n_neighbors=6),
        "features": X_gensim,
        "type": "knn",
    },
}

rec_eval = []
recommendations_output = {}

for name, cfg in recommenders.items():
    features = cfg["features"]
    # Xử lý an toàn cho model_type
    type_value = cfg.get("type", "knn")
    if type_value is None:
        model_type = "knn"
    else:
        model_type = str(type_value).strip().lower()
    
    if model_type == "cosine_matrix":
        # Content-based filtering với cosine similarity matrix
        # Convert sparse matrix to dense nếu cần
        if hasattr(features, "toarray"):
            features_dense = features.toarray()
        else:
            features_dense = features
        similarity_matrix = cosine_similarity(features_dense)
        
        sample_indices = rng.choice(features.shape[0], size=min(20, features.shape[0]), replace=False)
        distances_list = []
        for idx in sample_indices:
            similarities = similarity_matrix[idx]
            # Bỏ bản thân (similarity = 1.0), lấy top 5
            top_indices = np.argsort(similarities)[::-1][1:6]
            top_similarities = similarities[top_indices]
            # Chuyển similarity sang distance (1 - similarity)
            distances = 1 - top_similarities
            distances_list.append(distances)
        distances_arr = np.vstack(distances_list)
        
        # Ví dụ recommendations cho sample đầu tiên
        if name == "content_based_cosine_matrix":
            target_idx = 0
            similarities = similarity_matrix[target_idx]
            top_indices = np.argsort(similarities)[::-1][1:6]
            top_similarities = similarities[top_indices]
            recs = df_model.iloc[top_indices][["id", "Tiêu đề", "Thương hiệu", "Giá"]].copy()
            recs["similarity"] = top_similarities
            recs["distance"] = 1 - top_similarities
            recommendations_output[name] = recs
    
    else:
        # KNN models
        model = cfg.get("model")
        if model is None:
            print(f"Warning: Model for {name} is None, skipping...")
            continue
        model.fit(features)

        sample_indices = rng.choice(features.shape[0], size=min(20, features.shape[0]), replace=False)
        distances_list = []
        for idx in sample_indices:
            sample_vec = features[idx]
            if hasattr(sample_vec, "reshape"):
                sample_vec = sample_vec.reshape(1, -1)
            dists, inds = model.kneighbors(sample_vec, return_distance=True)
            distances_list.append(dists.flatten()[1:])  # bỏ bản thân
        distances_arr = np.vstack(distances_list)

        if name in ["knn_cosine_all", "gensim_doc2vec"]:
            target_idx = 0
            sample_vec = features[target_idx]
            if hasattr(sample_vec, "reshape"):
                sample_vec = sample_vec.reshape(1, -1)
            dists, inds = model.kneighbors(sample_vec, return_distance=True)
            recs = df_model.iloc[inds.flatten()[1:]][["id", "Tiêu đề", "Thương hiệu", "Giá"]].copy()
            recs["distance"] = dists.flatten()[1:]
            recommendations_output[name] = recs

    rec_eval.append(
        {
            "model": name,
            "description": cfg["description"],
            "avg_neighbor_distance": float(distances_arr.mean()),
            "std_neighbor_distance": float(distances_arr.std()),
        }
    )

rec_eval_df = pd.DataFrame(rec_eval)

# --- Clustering models -----------------------------------------------------
cluster_models = {
    "kmeans": {
        "description": "KMeans với k=5",
        "predict": lambda X: KMeans(n_clusters=5, random_state=RANDOM_SEED, n_init="auto").fit_predict(X),
    },
    "gmm": {
        "description": "Gaussian Mixture (diag covariance)",
        "predict": lambda X: GaussianMixture(n_components=5, covariance_type="diag", random_state=RANDOM_SEED).fit(X).predict(X),
    },
    "agglomerative": {
        "description": "Agglomerative clustering (Ward)",
        "predict": lambda X: AgglomerativeClustering(n_clusters=5, linkage="ward").fit_predict(X),
    },
}

cluster_results = []
cluster_summaries = {}

for name, cfg in cluster_models.items():
    labels = cfg["predict"](X_all)

    if len(np.unique(labels)) < 2:
        sil = float("nan")
        db = float("nan")
        ch = float("nan")
    else:
        sil = float(silhouette_score(X_all, labels))
        db = float(davies_bouldin_score(X_all, labels))
        ch = float(calinski_harabasz_score(X_all, labels))

    cluster_results.append(
        {
            "model": name,
            "description": cfg["description"],
            "silhouette": sil,
            "davies_bouldin": db,
            "calinski_harabasz": ch,
        }
    )

    summary = (
        pd.concat([df_model.reset_index(drop=True), pd.Series(labels, name="cluster")], axis=1)
        .groupby("cluster")
        .agg(
            count=("id", "count"),
            avg_price=("Giá", "mean"),
            median_year=("Năm đăng ký", "median"),
            top_brand=("Thương hiệu", lambda x: x.value_counts().head(3).to_dict()),
        )
        .sort_index()
    )
    cluster_summaries[name] = summary

cluster_eval_df = pd.DataFrame(cluster_results)

{
    "recommendation_eval": rec_eval_df,
    "sample_recommendations": recommendations_output,
    "cluster_eval": cluster_eval_df,
    "cluster_summaries": cluster_summaries,
}


{'recommendation_eval':                          model  \
 0               knn_cosine_all   
 1           knn_cosine_numeric   
 2               knn_tfidf_text   
 3  content_based_cosine_matrix   
 
                                          description  avg_neighbor_distance  \
 0           KNN (cosine) trên đặc trưng số + one-hot               0.025198   
 1        KNN (cosine) chỉ với đặc trưng số chuẩn hóa               0.000019   
 2           KNN (cosine) trên TF-IDF tiêu đề + mô tả               0.714231   
 3  Content-based filtering với cosine similarity ...               0.031460   
 
    std_neighbor_distance  
 0               0.069222  
 1               0.000051  
 2               0.079295  
 3               0.065890  ,
 'sample_recommendations': {'knn_cosine_all':         id                                           Tiêu đề Thương hiệu  \
  846    893     Xe Tay Ga Vespa Print Piaggio 2024 Đỏ Như Mới     Piaggio   
  3877  4104                                     xe Vespa

**Đánh giá mô hình**

- Bảng `recommendation_eval` chứa khoảng cách trung bình & độ lệch chuẩn giữa bản ghi và 5 hàng xóm gần nhất cho từng mô hình gợi ý (khoảng cách càng nhỏ → gợi ý càng sát). Có thể bổ sung thêm đánh giá thủ công/feedback người dùng nếu thu thập được.
- Bảng `cluster_eval` so sánh các mô hình phân cụm qua các chỉ số nội sinh: `silhouette` (cao hơn tốt hơn), `davies_bouldin` (thấp hơn tốt hơn), `calinski_harabasz` (cao hơn tốt hơn).
- `cluster_summaries` liệt kê thông tin thống kê cho từng cụm theo từng mô hình để phục vụ phân tích sâu hơn.



- Có thể thay `sample_index` bằng bất kỳ chỉ số nào để gợi ý cho xe khác.
- Lưu `recommendations` ra CSV để sử dụng trong ứng dụng.
- Với clustering, xem thêm biểu đồ phân cụm bằng cách giảm chiều (`PCA`/`UMAP`).
